In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import gc
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import time
import datetime
from datetime import datetime
import calendar

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans

sns.set_style('white')


pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
clientes_actuales = pd.read_pickle('/kaggle/input/easymoney/EasyMoney_base_segmentacion.pkl',compression='zip')

En este dataset ya tenemos los campos CLuster para la segmentacion con 6 grupos y Cluster2 para la de 5 grupos.

In [ ]:
clientes_actuales[['Cluster','Cluster2']]

In [ ]:
clientes_actuales.groupby(['Cluster']).agg({'Cluster2':[np.mean,np.min,np.max]})

La diferencia entre 6 o 5 grupos es que: 
1. Los grupos 0,3 y 5 pasan a ser los 0,3.
2. Los grupos 1,2 y 4 se quedan mas o menos como estan

# Vemos la agrupacion con 6 grupos:

Veamos un poco los grupos resultantes en esta segmentacion:

In [ ]:
clientes_actuales.groupby('Cluster').agg({
                                         'totalAssets':np.mean,
                                         'totalCuentas':np.mean,
                                         'totalAhorro':np.mean,
                                         'totalFinanciacion':np.mean,
                                         'salary':np.mean,
                                         'age':np.mean,
                                         'Cluster':len
                                        })

Vamos a pintar los counts totales de los productos Easy money por cluster

In [ ]:
# Easymoney product list
productos_easymoney=['loans',
 'mortgage',
 'funds',
 'securities',
 'long_term_deposit',
 'em_account_pp',
 'credit_card',
 'payroll',
 'pension_plan',
 'payroll_account',
 'emc_account',
 'debit_card',
 'em_account_p',
 'em_acount',
 'short_term_deposit']

Para esto hacemos primero una pivoty table, agrupando por fecha y cluster:

In [ ]:
clientes_actuales_=clientes_actuales.pivot_table(index=['pk_partition','Cluster'], 
                                                 values=productos_easymoney,
                                                aggfunc=[np.sum])

Cambiamos el nombre a las columnas, ya que no me gusta el doble indexado.

In [ ]:
clientes_actuales_.columns=['credit_card',
                            'debit_card','em_account_p','em_account_pp',
                            'em_acount','emc_account','funds','loans',
                            'long_term_deposit','mortgage','payroll','payroll_account',
                            'pension_plan','securities','short_term_deposit']

Resetamos el indice, sin borrar el antiguo ya que lo necesitamos.

In [ ]:
clientes_actuales_.reset_index(drop=False,inplace=True)

In [ ]:
clientes_actuales_

Hacemos un Melt de la tabla y agrupamos los datos de todos los productos Easymoney en solo dos columnas:

In [ ]:
clientes_actuales_=clientes_actuales_.melt(id_vars=['pk_partition','Cluster'],
              value_vars=productos_easymoney,
              var_name='Product',
              value_name='Count')

In [ ]:
clientes_actuales_

Lo pintamos:

In [ ]:
evolucion_horizontal = px.bar(clientes_actuales_, 
                              x="Cluster", y="Count", color='Product', orientation='v', 
                              height=600,title='Productos por Cluster', 
                              color_discrete_sequence = px.colors.cyclical.mygbm)
evolucion_horizontal.show()

In [ ]:
clientes_actuales.groupby('Cluster').agg({
                                         'totalAssets':np.mean,'totalCuentas':np.mean,'totalAhorro':np.mean,
                                         'totalFinanciacion':np.mean,'salary':np.mean,'age':np.mean,
                                         'Cluster':len})

Vamos a añadir una nueva variable para recoger la recomendacion. 
Esta columna sera un numero:

Productos:	Tipo:	Id Recomendación:

* debit_card	Cuenta simple	1
* em_account_p	Cuenta simple	2
* em_account_pp	Cuenta simple	3
* em_acount       Cuenta simple	4
* emc_account	Cuenta simple	5
* payroll		Cuenta simple	6
* payroll_account	Cuenta simple	7
* funds		Ahorro/inversion 8
* long_term_deposit	Ahorro/inversion 9
* mortgage	Ahorro/inversion 	10
* pension_plan	Ahorro/inversion 	11
* securities	Ahorro/inversion	12
* short_term_deposit	Ahorro/inversion 13
* loans		Financiacion	14
* credit_card	Financiacion	15


In [ ]:
clientes_actuales['recomendacion']=0

# Estudiamos el grupo 0

Grupo formado por 36000 clientes donde en promedio solo tienen dos productos EasyMoney y estos dos son de cuenta simple. Con un promedio de edad de 37 años y unos ingresos medio/bajos

Vemos que casi todos son productos de cuenta simple:

In [ ]:
clientes_actuales_[clientes_actuales_['Cluster']==0]

In [ ]:
evolucion_horizontal = px.bar(clientes_actuales_[clientes_actuales_['Cluster']==0], 
                              x="Cluster", y="Count", color='Product', orientation='v', 
                              height=600,title='Productos para Cluster 0', 
                              color_discrete_sequence = px.colors.cyclical.mygbm)
evolucion_horizontal.show()

**Podemos proponer a la gente de este grupo que tiene cuenta em_acount y no tiene debit_card, que contraten una debit_card**

Clientes con em_acount y debit_card:

In [ ]:
clientes_actuales[ (clientes_actuales['Cluster']==0) &
                   (clientes_actuales['em_acount']==1) &
                   (clientes_actuales['debit_card']==1) ]

Clientes con em_acount y sin debit_card:

In [ ]:
clientes_actuales[ (clientes_actuales['Cluster']==0) &
                   (clientes_actuales['em_acount']==1) &
                   (clientes_actuales['debit_card']==0)  ][['pk_cid','em_acount','debit_card']]

# Estudiamos el grupo 1

Grupo formado por 18000 clientes, con una media de edad de 39 años y unos ingresos altos. Tienen en promedio casi 4 productos EasyMoney de los que casi 3 son de cuenta simple y uno de ahorro.

In [ ]:
clientes_actuales_[clientes_actuales_['Cluster']==1]

In [ ]:
evolucion_horizontal = px.bar(clientes_actuales_[clientes_actuales_['Cluster']==1], 
                              x="Cluster", y="Count", color='Product', orientation='v', 
                              height=600,title='Productos para Cluster 1', 
                              color_discrete_sequence = px.colors.cyclical.mygbm)
evolucion_horizontal.show()

En este grupo hay 8400 clientes que tienen estos 4 productos: 'payroll_account','payroll','pension_plan','debit_card'

In [ ]:
clientes_actuales[ (clientes_actuales['Cluster']==1) &
                   (clientes_actuales['payroll']==1) &
                   (clientes_actuales['debit_card']==1) &
                   (clientes_actuales['pension_plan']==1) &
                   (clientes_actuales['payroll_account']==1)  ][['pk_cid','payroll_account','payroll','pension_plan','debit_card']]

Vamos a ofrecer a todos los clientes de este grupo que no tengan plan de pensiones, que lo contraten. Son unos 2776 clientes.

In [ ]:
len(clientes_actuales[ (clientes_actuales['Cluster']==1) &
                   (clientes_actuales['pension_plan']==0)  ])

In [ ]:
clientes_actuales.loc[ (clientes_actuales['Cluster']==1) &
                       (clientes_actuales['pension_plan']==0)  
                      ,'recomendacion']=11

Vemos tambien , los que no tienen payroll account: 

In [ ]:
len(clientes_actuales[ (clientes_actuales['Cluster']==1) &
                   (clientes_actuales['payroll_account']==0)  ])